###### **Here, we demonsrate how to collect the predictions of several models to generate an ensemble-averaged prediction on a given image.**

See: https://drive.google.com/file/d/1ku8X8lHs6lethEa5Adhj7frzV44NTbl4/view for methodological details.

Let's assume that for a given image I, you are given a maximum of 4 .txt files, each of which corresponds to the predictions from the 4 different models. Each .txt file will contain a list of bounding boxes in the following format:

c, s, xmin, ymin, xmax, ymax

where c = predicted class, s = confidence score and the last 4 numbers are the bounds of the bounding box. Now, say we want to create an ensemble prediction from these 4 .txt files. Note that there may be less than 4 .txt files if a given model predicts no objects for the image under consideration.

Thus, based on the notation of the paper, LD = [D1, D2, D3, D4], where each D_i = {d_ik} = {c_ik, s_ik, xmin_ik, ymin_ik, xmax_ik, ymax_ik}.

We use the affirmative strategy as outlined in the above paper followed by non-max suppression (NMS) to generate the ensemble-averaged prediction. The procedure is as follows:

###### We first create a list of all detections by flattening LD to get F = {d_j}, where j runs from 1 to the total number of detections (d_ik) from all D_i's combined.

In [45]:
import os
from collections import deque
import shutil

In [27]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines

In [28]:
def parse_line(line):
    # Split the line and convert relevant parts to appropriate data types
    data = line.split()
    category_label = data[0]
    confidence_score = float(data[1])
    xmin, ymin, xmax, ymax = map(float, data[2:])
    return category_label, confidence_score, xmin, ymin, xmax, ymax

In [29]:
def collect_data_from_files(file_paths):
    all_data = []
    for file_path in file_paths:
        lines = read_file(file_path)
        for line in lines:
          data_tuple = parse_line(line)
          all_data.append(data_tuple)
    return all_data

In [30]:
source_dir = '/content/drive/MyDrive/EYOpenScienceDataChallenge/code/ensembling'
file_paths = [os.path.join(source_dir, 'model_unR.txt'),
              os.path.join(source_dir, 'model_unC.txt'),
              os.path.join(source_dir, 'model_dR.txt'),
              os.path.join(source_dir, 'model_dC.txt')]
all_data_f = collect_data_from_files(file_paths)

##### We now convert the list F (stored above in the variable all_data_f) to a list G (to be stored in a variable all_data_g). This list G will contain detections {D_g_i}, where each Dg_i is itself a list such that D_g_i = {d_g_ik} = {c_g_ik, s_g_ik, xmin_g_ik, ymin_g_ik, xmax_g_ik, ymax_g_ik} and such that for any two elements of a given D_g_i, say d_g_im and d_g_im, we have:

- c_g_im = c_g_in
- IoU(b_g_im, b_g_in) > iou_thr_ensemble

where b_g_ik denotes the bounding box coordinates {xmin_g_ik, ymin_g_ik, xmax_g_ik, ymax_g_ik}.

In [19]:
def iou_compute(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [23]:
all_data_g = []
sublist_g = []
iou_thr_ensemble = 0.9
while all_data_f:
  all_data_f_deque = deque(all_data_f)
  all_data_f = []
  boxPrim = all_data_f_deque.popleft()
  sublist_g.append(boxPrim)
  while all_data_f_deque:
    box = all_data_f_deque.popleft()
    if boxPrim[0] == box[0] and iou_compute([boxPrim[2], boxPrim[3], boxPrim[4], boxPrim[5]], [box[2], box[3], box[4], box[5]]) > iou_thr_ensemble:
      sublist_g.append(box)
    else:
      all_data_f.append(box)
  all_data_g.append(sublist_g)
  sublist_g = []

We now use the 'Affirmative' strategy, which means that we leave the list all_data_g untouched. At this point, if you need to use some other strategy such as the 'Consensus' strategy or the 'Unanimous' strategy, then you need to write a separate code to post-process the all_data_g further.

##### The last step is to apply NMS to get our output set of ensemble predictions in a single list, called ensemble_predictions. However, in contrast to the paper, we do not apply NMs to each sublist within all_data_g. Instead, we apply NMS to the full data within all_data_g. In this way, especially in conjunction with the 'Affirmative' approach, we can directly work with all_data_f and apply NMS on top of it to get ensemble_predictions!

In [31]:
def nonMaximumSuppression(boxes, overlapThresh):
  sorted_boxes = sorted(boxes, key=lambda x: x[1], reverse=True)
  ensemble_predictions = []
  while sorted_boxes:
    sorted_boxes_deque = deque(sorted_boxes)
    sorted_boxes = []
    final_box = sorted_boxes_deque.popleft()
    ensemble_predictions.append(final_box)
    while sorted_boxes_deque:
      box = sorted_boxes_deque.popleft()
      if iou_compute([final_box[2], final_box[3], final_box[4], final_box[5]], [box[2], box[3], box[4], box[5]]) > overlapThresh:
        continue
      else:
        sorted_boxes.append(box)
  return ensemble_predictions

In [32]:
ensemble_predictions = nonMaximumSuppression(all_data_f, 0.9)

##### Now, let's combine the above workflow to process the ensembling for a collection of images!

In [36]:
# !unzip /content/drive/MyDrive/EYOpenScienceDataChallenge/code/ensembling/model_development_6/submission_1.zip

In [39]:
# !mv Val*txt /content/drive/MyDrive/EYOpenScienceDataChallenge/code/ensembling/model_development_6/

In [44]:
# Doing this for a case where predictions were generated from a single model trained on all 4 classes

source_dir = '/content/drive/MyDrive/EYOpenScienceDataChallenge/code/ensembling/model_development_6'
for filename in os.listdir(source_dir):
  if filename.endswith(".txt"):
    all_data_f = collect_data_from_files([os.path.join(source_dir, filename)])
    ensemble_predictions = nonMaximumSuppression(all_data_f, 0.9)

    outdir = '/content/drive/MyDrive/EYOpenScienceDataChallenge/code/ensembling/model_development_6/submission_2'
    if not os.path.exists(outdir):
      os.makedirs(outdir)
    output_file_path = os.path.join(outdir, filename)

    with open(output_file_path, 'w') as output_file:
      for data_tuple in ensemble_predictions:
        # Get coordinates of each bounding box
        class_names, confidences, left, top, right, bottom = data_tuple
        # Write content to file in desired format
        output_file.write(f"{class_names} {confidences} {left} {top} {right} {bottom}\n")

In [46]:
# Define your source directory and the destination where the zip file will be created
source_dir = '/content/drive/MyDrive/EYOpenScienceDataChallenge/code/ensembling/model_development_6/submission_2'
destination_zip = 'submission'

# Create a zip file from the directory
shutil.make_archive(destination_zip, 'zip', source_dir)

print(f"Directory {source_dir} has been successfully zipped into {destination_zip}.")

Directory /content/drive/MyDrive/EYOpenScienceDataChallenge/code/ensembling/model_development_6/submission_2 has been successfully zipped into submission.


In [47]:
!ls

drive  sample_data  submission.zip
